In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install vaderSentiment
!pip install afinn
#!pip install tabula-py
#!pip install -q tabula-py
!pip install PyPDF2
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53430 sha256=518e78f951c20dce9cba14b9890e21bb31b2ff433fba9cbce8019ffe96767437
  Stored in directory: /root/.cache/pip/wheels/b0/05/90/43f79196199a138fb486902fceca30a2d1b5228e6d2db8eb90
Successfully built afinn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string
#import tabula
#from tabula import read_pdf
from google.colab import drive
import PyPDF2
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving NEULAND.xlsx to NEULAND.xlsx


In [ ]:
import io
concall = pd.read_excel(io.BytesIO(uploaded['NEULAND.xlsx']))
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,0,NaN,740,844.0,NaN
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,0,NaN,844,952.0,740.0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,0,NaN,952,936.0,844.0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,0,NaN,936,1574.0,952.0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,0,NaN,1574,1377.0,936.0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,0,NaN,1377,1180.0,1574.0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,0,NaN,1180,852.0,1377.0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,0,NaN,852,718.0,1180.0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,0,NaN,718,566.0,852.0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,0,NaN,566,581.0,718.0


In [ ]:
#from tabula import read_pdf
#from tabula import wrapper
#df = wrapper.read_pdf('my_pdf')
#df = wrapper.read_pdf('/content/drive/MyDrive/concall/Neuland-Q3-FY21.pdf')
#dfs = tabula.read_pdf('/content/drive/MyDrive/concall/Neuland-Q4-FY21.pdf', pages="all", stream = True)
#dfs[0]
# convert PDF into CSV file
#tabula.convert_into('/content/drive/MyDrive/concall/Neuland-Q4-FY21.pdf', "/content/drive/MyDrive/concall/output.csv", output_format="csv", pages='all')

In [ ]:
#Use if not importing data frame excel
# assign data of lists.
#data = {'PDF': ['1 Neuland-Q3-FY21.pdf', '2 Neuland-Q4-FY21.pdf','3 Neuland-Q1-FY22.pdf','4 Neuland-Q2-FY22.pdf','5 Neuland-Q3-FY22.pdf'],'Company': ['Neuland', 'Neuland','Neuland', 'Neuland','Neuland'],'Sector': ['Pharma', 'Pharma','Pharma', 'Pharma','Pharma'], 'Year': ['2021', '2021','2022','2022','2022'],'Quarter': ['Q3', 'Q4','Q1','Q2','Q3'],'Page_Count': [0,0,0,0,0],'Content': ['','','','',''],'Stock_price': [1221,2425,2200,1823,1690]}
# Create DataFrame
#concall = pd.DataFrame(data)

In [ ]:
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,0,NaN,740,844.0,NaN
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,0,NaN,844,952.0,740.0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,0,NaN,952,936.0,844.0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,0,NaN,936,1574.0,952.0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,0,NaN,1574,1377.0,936.0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,0,NaN,1377,1180.0,1574.0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,0,NaN,1180,852.0,1377.0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,0,NaN,852,718.0,1180.0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,0,NaN,718,566.0,852.0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,0,NaN,566,581.0,718.0


In [ ]:
# Sample code already present in for loop
#pdfFileObj = open('/content/drive/MyDrive/concall/Neuland-Q3-FY21.pdf', 'rb')
#pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
# printing number of pages in pdf file
#page_count = pdfReader.numPages
# creating a page object
#pageObj = pdfReader.getPage(2)
# extracting text from page
#df = pageObj.extractText()
#df1 = ""
#for page in range(pdfReader.numPages):
#        pageObj = pdfReader.getPage(page)
#        df = pageObj.extractText()
#        df1 = df1+df
# closing the pdf file object
#pdfFileObj.close()

In [ ]:
mypath = '/content/drive/MyDrive/Con_call/'
from os import listdir
from os.path import isfile, join
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
length = len(files)

In [ ]:
pdfFileObj = open(mypath+files[0], 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
page_count = len(pdfReader.pages)
page_count
pageObj = pdfReader.pages[10]
pageObj

{'/Type': '/Page',
 '/Parent': {'/Type': '/Pages',
  '/Count': 14,
  '/Kids': [IndirectObject(3, 0, 138936197458688),
   IndirectObject(25, 0, 138936197458688),
   IndirectObject(31, 0, 138936197458688),
   IndirectObject(33, 0, 138936197458688),
   IndirectObject(35, 0, 138936197458688),
   IndirectObject(37, 0, 138936197458688),
   IndirectObject(39, 0, 138936197458688),
   IndirectObject(41, 0, 138936197458688),
   IndirectObject(43, 0, 138936197458688),
   IndirectObject(45, 0, 138936197458688),
   IndirectObject(47, 0, 138936197458688),
   IndirectObject(49, 0, 138936197458688),
   IndirectObject(51, 0, 138936197458688),
   IndirectObject(53, 0, 138936197458688)]},
 '/Resources': {'/XObject': {'/Image11': {'/Type': '/XObject',
    '/Subtype': '/Image',
    '/Width': 145,
    '/Height': 93,
    '/ColorSpace': '/DeviceRGB',
    '/BitsPerComponent': 8,
    '/Interpolate': False,
    '/Filter': '/FlateDecode'}},
  '/ExtGState': {'/GS7': {'/Type': '/ExtGState', '/BM': '/Normal', '/ca':

In [ ]:
pdfFileObj = open(mypath+files[0], 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
# printing number of pages in pdf file
page_count = len(pdfReader.pages)
# creating a page object
pageObj = pdfReader.pages[0]
pageObj

{'/Type': '/Page',
 '/Parent': {'/Type': '/Pages',
  '/Count': 14,
  '/Kids': [IndirectObject(3, 0, 138936197660832),
   IndirectObject(25, 0, 138936197660832),
   IndirectObject(31, 0, 138936197660832),
   IndirectObject(33, 0, 138936197660832),
   IndirectObject(35, 0, 138936197660832),
   IndirectObject(37, 0, 138936197660832),
   IndirectObject(39, 0, 138936197660832),
   IndirectObject(41, 0, 138936197660832),
   IndirectObject(43, 0, 138936197660832),
   IndirectObject(45, 0, 138936197660832),
   IndirectObject(47, 0, 138936197660832),
   IndirectObject(49, 0, 138936197660832),
   IndirectObject(51, 0, 138936197660832),
   IndirectObject(53, 0, 138936197660832)]},
 '/Resources': {'/Font': {'/F1': {'/Type': '/Font',
    '/Subtype': '/TrueType',
    '/Name': '/F1',
    '/BaseFont': '/TimesNewRomanPSMT',
    '/Encoding': '/WinAnsiEncoding',
    '/FontDescriptor': {'/Type': '/FontDescriptor',
     '/FontName': '/TimesNewRomanPSMT',
     '/Flags': 32,
     '/ItalicAngle': 0,
     '/As

In [ ]:
for i in range(length):
  pdfFileObj = open(mypath+files[i], 'rb')
  pdfReader = PyPDF2.PdfReader(pdfFileObj)
  # printing number of pages in pdf file
  page_count = len(pdfReader.pages)
  # creating a page object
  pageObj = pdfReader.pages[0]
  # extracting text from page
  df = pageObj.extract_text()
  df1 = ""
  for page in range(page_count):
        pageObj = pdfReader.pages[page]
        df = pageObj.extract_text()
        df1 = df1+df
        concall['PAGE_COUNT'][i] = page_count
        concall['CONTENT'][i] = df1
# closing the pdf file object
pdfFileObj.close()

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  concall['PAGE_COUNT'][i] = page_count
<ipython-input-18-bc76dcfcdfa3>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concall['PAGE_COUNT'][i] = page_count
<ipython-input-18-bc76dcfcdfa3>:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is whe

In [ ]:
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,14,\nPage 1 of 14 \n \n \n“Neuland Laboratories...,740,844.0,NaN
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,13,\n Page 1 of 13 \n \n \n“Neuland Laboratorie...,844,952.0,740.0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,13,Page 1 of 13 \n \n \n“Neuland Laboratories L...,952,936.0,844.0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,\n Page 1 of 15 \n \n \n“Neuland Laboratorie...,936,1574.0,952.0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,14,\n Page 1 of 14 \n \n \n“Neuland Laboratorie...,1574,1377.0,936.0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,10,\n Page 1 of 10 \n \n“Neuland Laboratories L...,1377,1180.0,1574.0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,24,\n Page 1 of 24 \n \n \n“Neuland Laboratorie...,1180,852.0,1377.0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,16,\n Page 1 of 16 \n \n \n“Neuland Laboratorie...,852,718.0,1180.0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,16,Page 1 of 16 \n \n \n“Neuland Laboratories Q...,718,566.0,852.0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,14,\n Page 1 of 14 \n \n \n“Neuland Laboratorie...,566,581.0,718.0


In [ ]:
def word_count(text_string):
  return len(text_string.split())

In [ ]:
concall['WORD_COUNT'] = concall['CONTENT'].apply(word_count)
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,14,\nPage 1 of 14 \n \n \n“Neuland Laboratories...,740,844.0,NaN,7084
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,13,\n Page 1 of 13 \n \n \n“Neuland Laboratorie...,844,952.0,740.0,6630
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,13,Page 1 of 13 \n \n \n“Neuland Laboratories L...,952,936.0,844.0,7074
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,\n Page 1 of 15 \n \n \n“Neuland Laboratorie...,936,1574.0,952.0,7527
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,14,\n Page 1 of 14 \n \n \n“Neuland Laboratorie...,1574,1377.0,936.0,7563
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,10,\n Page 1 of 10 \n \n“Neuland Laboratories L...,1377,1180.0,1574.0,4681
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,24,\n Page 1 of 24 \n \n \n“Neuland Laboratorie...,1180,852.0,1377.0,8348
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,16,\n Page 1 of 16 \n \n \n“Neuland Laboratorie...,852,718.0,1180.0,8340
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,16,Page 1 of 16 \n \n \n“Neuland Laboratories Q...,718,566.0,852.0,7911
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,14,\n Page 1 of 14 \n \n \n“Neuland Laboratorie...,566,581.0,718.0,6760


In [ ]:
#Identify and remove common words
freq = pd.Series(' '.join(concall['CONTENT']).split()).value_counts()[:20]
freq
freq = list(freq.index)
concall['CONTENT'] = concall['CONTENT'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
concall['CONTENT'].head()

,CONTENT
0,Page 1 14 “Neuland Laboratories Limited Q4 FY ...
1,Page 1 13 “Neuland Laboratories Limited Q1 FY1...
2,Page 1 13 “Neuland Laboratories Limited Q3 FY1...
3,Page 1 15 “Neuland Laboratories Limited Q2 FY2...
4,Page 1 14 “Neuland Laboratories Q1 FY20 Earnin...


In [ ]:
#Remove rare words
freq = pd.Series(' '.join(concall['CONTENT']).split()).value_counts()[-10:]
freq
freq = list(freq.index)
concall['CONTENT'] = concall['CONTENT'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
concall['CONTENT'].head()

,CONTENT
0,Page 1 14 “Neuland Laboratories Limited Q4 FY ...
1,Page 1 13 “Neuland Laboratories Limited Q1 FY1...
2,Page 1 13 “Neuland Laboratories Limited Q3 FY1...
3,Page 1 15 “Neuland Laboratories Limited Q2 FY2...
4,Page 1 14 “Neuland Laboratories Q1 FY20 Earnin...


In [ ]:
#Spelling correction - will take longer time to run. Avoid
#from textblob import TextBlob
#concall['Content'].apply(lambda x: str(TextBlob(x).correct()))

In [ ]:
#Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
concall['CONTENT'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

,CONTENT
0,page 1 14 “neuland laboratori limit q4 fy 2019...
1,page 1 13 “neuland laboratori limit q1 fy18 ea...
2,page 1 13 “neuland laboratori limit q3 fy19 ea...
3,page 1 15 “neuland laboratori limit q2 fy20 ea...
4,page 1 14 “neuland laboratori q1 fy20 earn con...
5,page 1 10 “neuland laboratori limit q3 fy20 ea...
6,page 1 24 “neuland laboratori limit q4 fy18 ea...
7,page 1 16 “neuland laboratori limit q2 fy21 ea...
8,page 1 16 “neuland laboratori q2 fy18 confer c...
9,page 1 14 “neuland laboratori ltd. q2 fy17 ear...


In [ ]:
concall['WORD_COUNT'] = concall['CONTENT'].apply(word_count)
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,14,Page 1 14 “Neuland Laboratories Limited Q4 FY ...,740,844.0,NaN,4880
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,13,Page 1 13 “Neuland Laboratories Limited Q1 FY1...,844,952.0,740.0,4495
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,13,Page 1 13 “Neuland Laboratories Limited Q3 FY1...,952,936.0,844.0,4784
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,936,1574.0,952.0,5049
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,14,Page 1 14 “Neuland Laboratories Q1 FY20 Earnin...,1574,1377.0,936.0,5131
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,1377,1180.0,1574.0,3233
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,24,Page 1 24 “Neuland Laboratories Limited Q4 FY1...,1180,852.0,1377.0,5657
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,852,718.0,1180.0,5599
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,16,Page 1 16 “Neuland Laboratories Q2 FY18 Confer...,718,566.0,852.0,5397
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,14,Page 1 14 “Neuland Laboratories Ltd. Q2 FY17 E...,566,581.0,718.0,4682


In [ ]:
#count stopwords
import nltk
nltk.download('all')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]    |   Unzipping corpora/ycoe.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection all


In [ ]:
stop_words = set(stopwords.words('english'))
concall['STOPWORDS_COUNT'] = concall['CONTENT'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT,STOPWORDS_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,14,Page 1 14 “Neuland Laboratories Limited Q4 FY ...,740,844.0,NaN,4880,983
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,13,Page 1 13 “Neuland Laboratories Limited Q1 FY1...,844,952.0,740.0,4495,952
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,13,Page 1 13 “Neuland Laboratories Limited Q3 FY1...,952,936.0,844.0,4784,991
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,936,1574.0,952.0,5049,924
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,14,Page 1 14 “Neuland Laboratories Q1 FY20 Earnin...,1574,1377.0,936.0,5131,1193
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,1377,1180.0,1574.0,3233,629
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,24,Page 1 24 “Neuland Laboratories Limited Q4 FY1...,1180,852.0,1377.0,5657,1199
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,852,718.0,1180.0,5599,1243
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,16,Page 1 16 “Neuland Laboratories Q2 FY18 Confer...,718,566.0,852.0,5397,1238
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,14,Page 1 14 “Neuland Laboratories Ltd. Q2 FY17 E...,566,581.0,718.0,4682,928


In [ ]:
#Remove stopwords
concall['CONTENT'] = concall['CONTENT'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
concall['WORD_COUNT'] = concall['CONTENT'].apply(word_count)
concall['STOPWORDS_COUNT'] = concall['CONTENT'].apply(lambda x: len([x for x in x.split() if x in stop_words]))
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT,STOPWORDS_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,14,Page 1 14 “Neuland Laboratories Limited Q4 FY ...,740,844.0,NaN,3897,0
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,13,Page 1 13 “Neuland Laboratories Limited Q1 FY1...,844,952.0,740.0,3543,0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,13,Page 1 13 “Neuland Laboratories Limited Q3 FY1...,952,936.0,844.0,3793,0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,936,1574.0,952.0,4125,0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,14,Page 1 14 “Neuland Laboratories Q1 FY20 Earnin...,1574,1377.0,936.0,3938,0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,1377,1180.0,1574.0,2604,0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,24,Page 1 24 “Neuland Laboratories Limited Q4 FY1...,1180,852.0,1377.0,4458,0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,852,718.0,1180.0,4356,0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,16,Page 1 16 “Neuland Laboratories Q2 FY18 Confer...,718,566.0,852.0,4159,0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,14,Page 1 14 “Neuland Laboratories Ltd. Q2 FY17 E...,566,581.0,718.0,3754,0


In [ ]:
concall['CONTENT'][0]

'Page 1 14 “Neuland Laboratories Limited Q4 FY 2019 Earnings Conference Call ” May 17 , 2019 MANAGEMENT : MS. SHIVANGI BUBNA - CHRISTENSEN IR MR. DIWAKAR PINGLE - CHRISTENSEN IR MR. DAVULURI SUCHETH RAO - VICE-CHAIRMAN AND CHIEF EXECUTIVE OFFICER - NEULAN D LABORATORIES LIMITED MR. DAVULURI SAHARSH RAO - JOINT MANAGING DIRECTOR - NEULAND LABORATORIES LIMITED MR. AMIT AGARWAL – CHIEF FINANCIAL OFFICER - NEULAND LABORATORIES LIMITED Neuland Laborato ries Limited May 17, 2019 Page 2 14 Moderator: Good day welcome Neuland Laboratories Q4 FY219 Earnings C onference Call. As reminder, participant lines listen -only mode opportunity ask questions presentation concludes. Should need assistance conference call, please signal operato r pressing “*” “0” touchtone phone. Please note conference recorded. hand conference Ms. Shivangi Bubna Christensen IR . Thank you, Madam! Shivangi Bubna : Thank Aman. Go od morning thank joining fourth quarter full year FY2019 earnings call Neuland Laboratories Lim

In [ ]:
analyzer = SentimentIntensityAnalyzer()
def sentiment_analyzser_scores(text):
  score = analyzer.polarity_scores(text)
  print(text)
  print(score)

In [ ]:
# Some simple text analysis
text_pos = 'it was an excellent performance. Everyone did well'
sentiment_analyzser_scores(text_pos)

it was an excellent performance. Everyone did well
{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.7003}


In [ ]:
# Some added negative score for simple text analysis
text_pos = 'it was an excellent performance. Everyone did well. However, few students did bad'
sentiment_analyzser_scores(text_pos)

it was an excellent performance. Everyone did well. However, few students did bad
{'neg': 0.181, 'neu': 0.518, 'pos': 0.301, 'compound': 0.3182}


In [ ]:
text_pos = 'it could have been better'
sentiment_analyzser_scores(text_pos)

it could have been better
{'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.4404}


In [ ]:
# Score all 4000-5000 words contents
text_pos = concall['CONTENT'][1]
sentiment_analyzser_scores(text_pos)

Page 1 13 “Neuland Laboratories Limited Q1 FY18 Earnings Call” September 11, 2017 MANAGEMENT : MR. SUCHETH RAO - VICE CHAIRMAN & CEO, NEULAND LABORATORIES LIMITED MR. SAHARSH RAO - JOINT MANAGING DIRECTOR , NEULAND LABORATORIES LIMITED MR. KUMAR GUPTA - AVP (FINANCE ), NEULAND LABORATORIES LIMITED MS. SARADA BHAMIDIPATI - COMPANY SECRETARY , NEULAND LABORATORIES LIMITED Neuland Laboratories Limited September 11, 2017 Page 2 13 Moderator: Good day, l adies gentlemen warm welcome Neuland Laboratories Limited Q1 FY18 Earnings Conference Call. As reminder, participant lines listen -only mode. There opportunity ask questions presentation concludes. Should need ass istance conference call, may signal operator pressing * 0 touchtone phone. Please note conference recorded. hand conference Mr. Diwakar Pingle Christensen IR. Thank you, sir. Diwakar Pingle: Thanks, Ali. Welcome thanks joining Q1 FY18 earnings call Neuland Laboratories. Please note emailed press release everyone also see results w

In [ ]:
# lets say any compounded score of >0.5 is positive and less is negative
# --- depends on domain/context/ relativeness, etc
def get_pos_word(x):
  text = x['CONTENT']
  tokenized_text = nltk.word_tokenize(text)
  pos_word_list =[]
  neu_word_list=[]
  neg_word_list=[]
  for word in tokenized_text:
    if(analyzer.polarity_scores(word)['compound'])>=0.5:
      pos_word_list.append(word)
    elif (analyzer.polarity_scores(word)['compound']) <=-0.5:
      neg_word_list.append(word)
    else:
      neu_word_list.append(word)
  return set(pos_word_list)

In [ ]:
def get_neg_word(x):
  text = x['CONTENT']
  tokenized_text = nltk.word_tokenize(text)
  pos_word_list =[]
  neu_word_list=[]
  neg_word_list=[]
  for word in tokenized_text:
    if(analyzer.polarity_scores(word)['compound'])>=0.5:
      pos_word_list.append(word)
    elif (analyzer.polarity_scores(word)['compound']) <=-0.5:
      neg_word_list.append(word)
    else:
      neu_word_list.append(word)
  return set(neg_word_list)

In [ ]:
from afinn import Afinn
afinn = Afinn(language = 'en')

In [ ]:
concall['AFINN_SCORE'] = concall['CONTENT'].apply(afinn.score)

In [ ]:
# new column gets created called afinnscore
concall

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT,STOPWORDS_COUNT,AFINN_SCORE
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,14,Page 1 14 “Neuland Laboratories Limited Q4 FY ...,740,844.0,NaN,3897,0,367.0
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,13,Page 1 13 “Neuland Laboratories Limited Q1 FY1...,844,952.0,740.0,3543,0,221.0
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,13,Page 1 13 “Neuland Laboratories Limited Q3 FY1...,952,936.0,844.0,3793,0,283.0
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,936,1574.0,952.0,4125,0,464.0
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,14,Page 1 14 “Neuland Laboratories Q1 FY20 Earnin...,1574,1377.0,936.0,3938,0,359.0
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,1377,1180.0,1574.0,2604,0,226.0
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,24,Page 1 24 “Neuland Laboratories Limited Q4 FY1...,1180,852.0,1377.0,4458,0,324.0
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,852,718.0,1180.0,4356,0,502.0
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,16,Page 1 16 “Neuland Laboratories Q2 FY18 Confer...,718,566.0,852.0,4159,0,258.0
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,14,Page 1 14 “Neuland Laboratories Ltd. Q2 FY17 E...,566,581.0,718.0,3754,0,421.0


In [ ]:
# normalising is very important because it may be that score is high because
# of huge amount of pages of PDF
concall['AFINN_ADJUSTED'] = concall['AFINN_SCORE']/concall['WORD_COUNT'] * 100

In [ ]:
sentiment = concall['CONTENT'].apply(analyzer.polarity_scores)

In [ ]:
sentiment_df = pd.DataFrame(sentiment.tolist())
sentiment_df

,neg,neu,pos,compound
0,0.035,0.755,0.210,1.0000
1,0.036,0.798,0.166,0.9999
2,0.038,0.786,0.176,1.0000
3,0.032,0.773,0.196,1.0000
4,0.035,0.779,0.186,1.0000
5,0.038,0.773,0.190,0.9999
6,0.048,0.768,0.184,1.0000
7,0.034,0.761,0.205,1.0000
8,0.044,0.802,0.154,0.9999
9,0.016,0.793,0.190,1.0000


In [ ]:
df_sentiment = pd.concat([concall,sentiment_df], axis = 1)

In [ ]:
df_sentiment

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,STOCK_PRICE_LAG,WORD_COUNT,STOPWORDS_COUNT,AFINN_SCORE,AFINN_ADJUSTED,neg,neu,pos,compound
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,14,Page 1 14 “Neuland Laboratories Limited Q4 FY ...,740,844.0,NaN,3897,0,367.0,9.417501,0.035,0.755,0.210,1.0000
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,13,Page 1 13 “Neuland Laboratories Limited Q1 FY1...,844,952.0,740.0,3543,0,221.0,6.237652,0.036,0.798,0.166,0.9999
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,13,Page 1 13 “Neuland Laboratories Limited Q3 FY1...,952,936.0,844.0,3793,0,283.0,7.461113,0.038,0.786,0.176,1.0000
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,936,1574.0,952.0,4125,0,464.0,11.248485,0.032,0.773,0.196,1.0000
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,14,Page 1 14 “Neuland Laboratories Q1 FY20 Earnin...,1574,1377.0,936.0,3938,0,359.0,9.116303,0.035,0.779,0.186,1.0000
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,1377,1180.0,1574.0,2604,0,226.0,8.678955,0.038,0.773,0.190,0.9999
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,24,Page 1 24 “Neuland Laboratories Limited Q4 FY1...,1180,852.0,1377.0,4458,0,324.0,7.267833,0.048,0.768,0.184,1.0000
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,852,718.0,1180.0,4356,0,502.0,11.524334,0.034,0.761,0.205,1.0000
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,16,Page 1 16 “Neuland Laboratories Q2 FY18 Confer...,718,566.0,852.0,4159,0,258.0,6.203414,0.044,0.802,0.154,0.9999
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,14,Page 1 14 “Neuland Laboratories Ltd. Q2 FY17 E...,566,581.0,718.0,3754,0,421.0,11.214704,0.016,0.793,0.190,1.0000


In [ ]:
df_sentiment['POS_WORD'] = concall.apply(lambda x:get_pos_word(x),axis=1)

In [ ]:
df_sentiment['NEG_WORD'] = concall.apply(lambda x:get_neg_word(x),axis=1)

In [ ]:
df_sentiment['POS_COUNT'] = df_sentiment['POS_WORD'].apply(lambda x: len(x))

In [ ]:
df_sentiment['NEG_COUNT'] = df_sentiment['NEG_WORD'].apply(lambda x: len(x))

In [ ]:
df_sentiment

,FILE_NAME,COMPANY_NAME,SECTOR,YEAR,QUARTER,YEAR_QUARTER,PAGE_COUNT,CONTENT,STOCK_PRICE,STOCK_PRICE_LEAD,...,AFINN_SCORE,AFINN_ADJUSTED,neg,neu,pos,compound,POS_WORD,NEG_WORD,POS_COUNT,NEG_COUNT
0,1 Neuland-Q4-FY16.pdf,NEULAND LABORATORIES,PHARMA,FY16,Q4,FY16-Q4,14,Page 1 14 “Neuland Laboratories Limited Q4 FY ...,740,844.0,...,367.0,9.417501,0.035,0.755,0.210,1.0000,"{positive, kind, successful, happy, great, fre...","{violates, killed, rage}",10,3
1,2 Neuland-Q1-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q1,FY17-Q1,13,Page 1 13 “Neuland Laboratories Limited Q1 FY1...,844,952.0,...,221.0,6.237652,0.036,0.798,0.166,0.9999,"{kind, positive, appreciation, encouraging, ou...","{pain, negative, ass}",8,3
2,3 Neuland-Q2-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q2,FY17-Q2,13,Page 1 13 “Neuland Laboratories Limited Q3 FY1...,952,936.0,...,283.0,7.461113,0.038,0.786,0.176,1.0000,"{kind, positive, successful, happy, best, posi...","{fu, worst, bad, violating, punitive}",10,5
3,4 Neuland-Q3-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q3,FY17-Q3,15,Page 1 15 “Neuland Laboratories Limited Q2 FY2...,936,1574.0,...,464.0,11.248485,0.032,0.773,0.196,1.0000,"{comfortable, kind, pleasure, positive, succes...","{negative, worst}",11,2
4,5 Neuland-Q4-FY17.pdf,NEULAND LABORATORIES,PHARMA,FY17,Q4,FY17-Q4,14,Page 1 14 “Neuland Laboratories Q1 FY20 Earnin...,1574,1377.0,...,359.0,9.116303,0.035,0.779,0.186,1.0000,"{comfortable, kind, happy, encouraging, Trust,...","{bad, dead, negative}",13,3
5,6 Neuland-Q1-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q1,FY18-Q1,10,Page 1 10 “Neuland Laboratories Limited Q3 FY2...,1377,1180.0,...,226.0,8.678955,0.038,0.773,0.190,0.9999,"{comfortable, positive, kind, happy, encouragi...","{failure, bad}",7,2
6,7 Neuland-Q2-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q2,FY18-Q2,24,Page 1 24 “Neuland Laboratories Limited Q4 FY1...,1180,852.0,...,324.0,7.267833,0.048,0.768,0.184,1.0000,"{comfortable, kind, happy, great, super, free,...","{bad, lying}",11,2
7,8 Neuland-Q3-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q3,FY18-Q3,16,Page 1 16 “Neuland Laboratories Limited Q2 FY2...,852,718.0,...,502.0,11.524334,0.034,0.761,0.205,1.0000,"{trust, positive, kind, successful, best, grea...",{worst},13,1
8,9 Neuland-Q4-FY18.pdf,NEULAND LABORATORIES,PHARMA,FY18,Q4,FY18-Q4,16,Page 1 16 “Neuland Laboratories Q2 FY18 Confer...,718,566.0,...,258.0,6.203414,0.044,0.802,0.154,0.9999,"{pleasure, kind, happy, Trust, great, best, st...","{depression, disappointment, ass, worst}",7,4
9,10 Neuland-Q1-FY19.pdf,NEULAND LABORATORIES,PHARMA,FY19,Q1,FY19-Q1,14,Page 1 14 “Neuland Laboratories Ltd. Q2 FY17 E...,566,581.0,...,421.0,11.214704,0.016,0.793,0.190,1.0000,"{comfortable, positive, kind, happy, best, hop...","{fu, fail, war}",7,3


In [ ]:
df_sentiment.drop(['CONTENT'], axis =1, inplace = True)

In [ ]:
from google.colab import files
df_sentiment.to_csv("sentiment_analysis.csv",encoding="utf_8")
files.download('sentiment_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>